# GPU Numerical Computation

Como se mencionó anteriormente en el curso, las computaciones numéricas realizadas por gpu superan enormemente a las realizadas por cpu en términos de tiempo de ejecución y eficiencia

Para ser capaces de acceder a este tipo de computaciones, es necesario hacer uso de:

* Nvidia GPU
* CUDA y CUDNN
* PyTorch

Para hacer uso de gpus potentes requeridos por algoritmos poderosos, tenemos las siguientes opciones:

1. Usar Google Colab (GPU gratis y opciones de suscripción)
2. Nvidia GPU local (Requiere set-up con los drivers de Nvidia, además de CUDA y CUDNN)
3. Usar cloud computing (Servicio de suscripción como: AWS, GCP o Azure)

Para las opciones 2) y 3) es necesario hacer el set-up y configuraciones necesarias con Nvidia drivers, CUDA, CUDNN y PyTorch

Referencias:

* CUDA: https://developer.nvidia.com/cuda-downloads
* CUDNN: https://developer.nvidia.com/cudnn
* PyTorch: https://pytorch.org/

## Revisar Especificaciones:

In [1]:
!nvidia-smi

Thu Jan  2 15:02:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.05              Driver Version: 560.35.05      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4050 ...    On  |   00000000:01:00.0  On |                  N/A |
| N/A   44C    P5              4W /   35W |     433MiB /   6141MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

En mi caso, el gpu es: NVIDIA GeForce RTX 4050

## Revisar Acceso a GPU con PyTorch

Para revisar si CUDA está configurado correctamente y es posible utilizarlo para nuestros modelos se corre el siguiente comando:

In [2]:
import torch
torch.cuda.is_available()

True

## Set-Up Device Agnostic Code

Es importante crear código agnostico al device utilizado (CPU o GPU) debido a que en caso de que se tenga acceso a un GPU con CUDA deberíamos utilizarlo pero en caso contrario nuestro código debe correr en máquinas sin acceso a ello

La siguiente línea configura donde se correrá nuestro código. En caso de que detecte CUDA configurado correctamente se selecciona del GPU, de lo contrario se utiliza el CPU

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

Es importante mencionar que este método es utilizado con jupyter notebooks, para scripts de python se utiliza otro método

Revisar la documentación de CUDA y PyTorch, además de las buenas prácticas recomendadas: https://pytorch.org/docs/stable/notes/cuda.html

## Conteo de GPUs

Dependiendo el área de trabajo y el proyecto asociado es posible que se tenga a disposición múltiples GPUs con el propósito de correr paralelamente los algoritmos

Para revisar a cuantos se tiene acceso se utiliza el siguiente comando:

In [4]:
torch.cuda.device_count()

1

# Crear Modelos en el GPU

Debido a que queremos hacer uso de nuestro GPU para agilizar las operaciones tensoriales de nuestros modelos, podemos hacerlo de la siguiente forma:

In [6]:
tns = torch.tensor([1, 2, 3])
tns, tns.device

(tensor([1, 2, 3]), device(type='cpu'))

## Mover Tensores al GPU

Por defecto, nuestros tensores serán creados en el CPU

Es posible tanto crearlos directamente en el GPU, como moverlos del device utilizado al device deseado

Para mover un tensor A de device podemos: 

1. Hacer uso de `A.to(device="cuda")` 


2. Especificar previamente `device = "cuda" if torch.cuda.is_available() else "cpu"` y luego usar `A.to(device)`

In [10]:
gpu_tns = tns.to(device)
gpu_tns

tensor([1, 2, 3], device='cuda:0')

In [9]:
gpu_tns.device

device(type='cuda', index=0)

Debido a que solo tenemos acceso a 1 GPU, nos indica que nuestro tensor está en nuestro GPU con índice 0 (recordando que la numeración simpre inicia en 0). En caso de que tuvieramos acceso a múltiples GPUs es posible indicar que cual deseamos crear nuestros modelos

**Los modelos en GPU no pueden transformarse con Numpy**

## Regresar Tensores al CPU

Debido a que los tensores en GPU no pueden operar con Numpy, es importante controlar el device se tienen los tensores

Para ello, usamos en el tensor A (actualmente en el gpu) `A.cpu()` y lo guardamos en otra variable

In [11]:
cpu_tns = gpu_tns.cpu()
cpu_tns, cpu_tns.device

(tensor([1, 2, 3]), device(type='cpu'))

Una vez realizado el cambio de device, es posible operar con Numpy:

In [12]:
cpu_tns.numpy()

array([1, 2, 3])

Debido a que al realizar el cambio de device se hace una copia del tensor original en el nuevo device y éstas no comparten memoria, nuestro tensor original se mantiene en su device correspondiente:

In [14]:
gpu_tns, gpu_tns.device

(tensor([1, 2, 3], device='cuda:0'), device(type='cuda', index=0))